In [1]:
import pandas as pd

from pandas_datareader import data
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

In [2]:
fig_size = (12, 4)
start_date = '1970-01-02'
today = datetime.today().strftime('%Y-%m-%d')
one_year = (datetime.today()-relativedelta(years=1)).strftime('%Y-%m-%d')
five_years = (datetime.today()-relativedelta(years=5)).strftime('%Y-%m-%d')
ten_years = (datetime.today()-relativedelta(years=10)).strftime('%Y-%m-%d')

In [3]:
def get_dma_analysis(ticker, start_date=start_date, end_date=datetime.today().strftime('%Y-%m-%d'), ma_period=200):
    df = data.DataReader(ticker, 'yahoo', start_date, end_date).reset_index()
    df['DMA'] = df['Close'].rolling(ma_period).mean()
    df['PCH_DMA'] = 100*(df['Close']-df['DMA'])/df['DMA']
    df['PCH_DMA_MEAN'] = df['PCH_DMA'].mean()
    df['PCH_DMA_MEDIAN'] = df['PCH_DMA'].median()
    df = df.round(3)
#     print('Start', df[['Date', 'Close']].set_index('Date').head(1).transpose())
#     print('\nEnd', df[['Date', 'Close', 'DMA', 'PCH_DMA', 'PCH_DMA_MEAN', 'PCH_DMA_MEDIAN']].set_index('Date').tail(1).transpose())
    return df
        
def plot_dma_analysis(df, start_date=start_date, end_date=datetime.today().strftime('%Y-%m-%d'), title=''):
    df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].plot(
        x='Date', y=['Close', 'DMA'], style = ['-', '--'],
        figsize=fig_size, grid=True, title=title)
    df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].plot(
        x='Date', y=['PCH_DMA', 'PCH_DMA_MEAN', 'PCH_DMA_MEDIAN'], style=['-', '--', '--'],
        figsize=fig_size, grid=True, title=title)
    
def plot_dma_hist(df, bins=100, title=''):
    df.plot(
        x='Date', y=['PCH_DMA'], kind='hist', bins=bins,
        figsize=fig_size, grid=True, title=title)
    
def get_dma_percentile(df, bins=100):
    current_pch_dma_rank_percentile = round(100*df['PCH_DMA'].dropna().rank(axis=0).tail(1).values[0]/len(df['PCH_DMA'].dropna()), 2)
#     print('Current PCH DMA percentile:', current_pch_dma_rank_percentile)
    return(current_pch_dma_rank_percentile)

In [4]:
# tickers = ['AMZN', 'AAPL', 'FB', 'GOOGL', 'MSFT', 'UBER', 'XOM', 'BA', 'DAL', 'UAL', 'AAL', 'SMH', 'IVV',
#            'VNQ', 'VCSH', '^GSPC', 'ES=F', 'QQQ', 'SHV', '^VIX', '^TNX', 'CL=F', '^HSI', '^NSEI', 'MCHI',
#            'INDA', 'INR=X', 'GLD', 'BTC-USD', 'BRK-B', 'V', 'WMT', 'PG', 'JNJ', 'TSLA', 'PG', 'MA', 'UNH',
#            'HD', 'VZ', 'JPM', 'NVDA', 'AXP', 'SQ', 'CVS', 'GE', 'DIS', 'CRM', 'NFLX', 'BAC', 'KO', 'INTC', 'T', 'CVX', 'IBM', 'LMT', 'SBUX', 'AMD', 'MU', 'SPOT',
#            'TWTR', 'LYFT', 'BABA', 'TCEHY', 'TSM', 'XLK', 'XLV', 'XLC', 'XLY', 'XLF', 'XLI', 'XLP', 'XLU',
#            'XLE', 'XLRE', 'XLB', 'VGSH', 'BSV', 'BIV', 'BLV', 'BND']

tickers = ['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'TSLA', 'V', 'WMT', 'JNJ', 'JPM', 'PG', 'NVDA', 'MA', 'UNH',
           'HD', 'DIS', 'VZ', 'CRM', 'BAC', 'KO', 'PYPL', 'CMCSA', 'ADBE', 'BRK-A', 'NFLX', 'NKE', 'PFE',
           'MRK', 'T', 'PEP', 'ABT', 'INTC', 'TMO', 'ABBV', 'CSCO', 'COST', 'ORCL', 'QCOM', 'CVX', 'TMUS',
           'ACN', 'DHR', 'MCD', 'XOM', 'AVGO', 'MDT', 'NEE', 'TXN', 'HON', 'UPS', 'BMY', 'LLY', 'UNP', 'LIN',
           'CHTR', 'AMGN', 'ZM', 'PM', 'SBUX', 'LOW', 'BA', 'C', 'RTX', 'WFC', 'MS', 'IBM', 'AMT', 'LMT',
           'BLK', 'AMD', 'NOW', 'MMM', 'CAT', 'INTU', 'AXP', 'FIS', 'SQ', 'SYK', 'SCHW', 'EL', 'CVS', 'TGT',
           'ISRG', 'GE', 'UBER', 'MDLZ', 'BKNG', 'SPGI', 'DE', 'ZTS', 'GS', 'ANTM', 'FISV', 'GILD', 'MO',
           'PLD', 'CI', 'ADP', 'CL', 'TJX', 'FDX', 'CCI', 'AMAT', 'BX', 'CSX', 'MU', 'DUK', 'BDX',
           'CB', 'SNAP', 'SHW', 'EQIX', 'ITW', 'SO', 'USB', 'D', 'LRCX', 'TFC', 'VMW', 'GM', 'NSC', 'ECL',
           'CME', 'APD', 'ATVI', 'MMC', 'GPN', 'ICE', 'VRTX', 'ADSK', 'REGN', 'PNC', 'PGR', 'HUM', 'EW', 'DG',
           'TEAM', 'MCO', 'NOC', 'DELL', 'WM', 'HCA', 'ADI', 'WDAY', 'NEM', 'SPOT', 'BSX', 'TWLO', 'KMB',
           'ETN', 'DD', 'EMR', 'LULU', 'MNST', 'SCCO', 'ILMN', 'LVS', 'GD', 'ROP', 'COP', 'KDP', 'EPD',
           'VEEV', 'PINS', 'MET', 'DOCU', 'AEP', 'EXC', 'DOW', 'CTSH', 'CVNA', 'LHX', 'DLR', 'IDXX', 'KHC',
           'BAX', 'PSA', 'TTD', 'ROST', 'MAR', 'MRNA', 'COF', 'APH', 'CTAS', 'SRE', 'BIIB', 'GIS', 'KLAC',
           'INFO', 'TEL', 'CNC', 'XEL', 'CMG', 'SYY', 'TWTR', 'EA', 'ALGN', 'CSGP', 'MTCH', 'TT', 'PH', 'A',
           'F', 'CMI', 'MCHP', 'TRV', 'STZ', 'PPG', 'EBAY', 'BK', 'XLK', 'XLV', 'XLC', 'XLY', 'XLF', 'XLI',
           'XLP', 'XLU', 'XLE', 'XLRE', 'XLB', 'VCSH', 'VGSH', 'BSV', 'BIV', 'BLV', 'BND', 'QQQ', 'IVV', 'SHV',
           'CL=F', 'MCHI', 'INDA', 'GLD', 'BTC-USD', 'DAL', 'UAL', 'AAL', 'SMH']

In [5]:
a, b, c, d, e, f = ([] for i in range(6))

for ticker in tqdm(tickers):
    try:
        df = get_dma_analysis(ticker=ticker, start_date=start_date)
        a.append(get_dma_percentile(df))
        b.append(df[['PCH_DMA']].tail(1).values[0][0])
        c.append(df[['PCH_DMA_MEAN']].tail(1).values[0][0])
        d.append(df[['PCH_DMA_MEDIAN']].tail(1).values[0][0])
        e.append(df[['Close']].tail(1).values[0][0])
        f.append(df[['DMA']].tail(1).values[0][0])
    except:
        print(ticker)

In [6]:
df = pd.DataFrame()
df['ticker'] = tickers
df['percentile'] = a
df['pch'] = b
df['pch_mean'] = c
df['pch_median'] = d
df['close'] = e
df['dma'] = f

In [7]:
df.sort_values(by='percentile', ascending=True).head(15).transpose()

,229,93,172,175,66,138,56,222,185,148,67,128,43,31,111
ticker,SHV,GILD,DLR,BAX,AMT,NOC,ZM,VGSH,BIIB,KMB,LMT,VRTX,XOM,INTC,EQIX
percentile,0.05,10.28,10.8,13.21,15.28,18.37,18.4,18.94,19.49,21.08,21.47,22.1,23.63,24.15,25.78
pch,-0.223,-13.12,-8.077,-8.789,-6.303,-7.125,47.377,-0.224,-13.831,-2.699,-3.524,-11.619,-3.375,-7.623,-2.67
pch_mean,0.035,8.155,6.209,3.433,6.241,4.342,87.327,0.107,8.911,4.056,5.638,6.568,2.617,5.705,6.94
pch_median,0.011,6.359,6.515,4.951,7.844,5.102,86.89,0.038,6.285,4.062,5.75,3.274,2.8,4.539,9.006
close,110.51,61.13,132.16,76.35,228.9,298.77,412.86,61.95,246.92,138.34,361.36,226.7,39.94,49.9,690.62
dma,110.756,70.361,143.773,83.707,244.299,321.692,280.138,62.089,286.555,142.177,374.56,256.504,41.335,54.018,709.566


In [8]:
df.sort_values(by='percentile', ascending=False).head(15).transpose()

,83,84,200,88,119,232,179,215,212,238,109,188,100,228,220
ticker,GE,UBER,PH,DE,GM,INDA,MAR,XLI,XLC,SMH,SNAP,INFO,FDX,IVV,XLB
percentile,100,100,99.87,99.87,99.79,99.75,99.68,99.62,99.52,99.35,99.2,99.16,99.13,98.95,98.76
pch,42.848,57.865,43.668,42.447,55.921,21.793,34.603,21.989,20.801,36.205,105.955,23.445,60.045,16.4,21.414
pch_mean,2.289,3.657,3.998,3.891,0.554,1.76,4.826,2.081,6.069,2.699,10.571,8.15,5.242,1.965,1.97
pch_median,3.839,4.143,4.816,4.373,0.398,2.129,6.769,3.704,6.759,4.773,11.239,8.426,4.086,3.941,3.594
close,10.43,53.09,270.11,259,44.58,37.51,128.72,87.93,67.22,213.25,47.49,92.55,291.72,368.1,70.85
dma,7.301,33.63,188.01,181.822,28.591,30.798,95.629,72.08,55.645,156.566,23.058,74.972,182.273,316.237,58.354


In [9]:
df.sort_values(by='pch_median', ascending=False).head(15).transpose()

,56,180,177,170,147,76,165,136,24,163,70,234,162,11,192
ticker,ZM,MRNA,TTD,CVNA,TWLO,SQ,DOCU,TEAM,NFLX,PINS,NOW,BTC-USD,VEEV,NVDA,CMG
percentile,18.4,90.07,97.9,75.85,75.24,93.08,57.77,79.55,40.7,92.92,84.03,84.65,55.85,75.62,67.41
pch,47.377,131.51,103.831,57.705,53.122,70.147,30.765,31.419,10.531,117.886,28.437,63.826,18.401,33.487,22.303
pch_mean,87.327,56.757,33.279,36.457,25.023,27.25,35.212,19.608,17.431,28.791,14.555,22.79,15.475,13.72,10.142
pch_median,86.89,48.453,35.25,31.127,29.137,28.23,27.191,19.979,17.51,17.277,15.447,15.275,14.843,13.256,12.71
close,412.86,143,866.92,225.45,320.34,202,217.7,227.99,503.38,68.01,526.52,18989.5,272.4,541.78,1297
dma,280.138,61.768,425.313,142.957,209.205,118.721,166.481,173.483,455.421,31.214,409.946,11591.3,230.065,405.866,1060.48


In [10]:
df.sort_values(by='pch_median', ascending=True).head(15).transpose()

,174,168,237,194,202,229,222,223,221,119,226,224,69,144,219
ticker,KHC,DOW,AAL,TWTR,F,SHV,VGSH,BSV,VCSH,GM,BND,BIV,AMD,NEM,XLRE
percentile,84.13,95.28,67.69,89.75,96.24,0.05,18.94,46.57,84.98,99.79,41.88,49.61,86.6,46.47,70.88
pch,7.12,26.962,13.87,30.464,40.368,-0.223,-0.224,0.052,1.35,55.921,0.368,0.913,43.413,-0.024,4.342
pch_mean,-6.526,-3.591,-0.31,0.7,1.845,0.035,0.107,0.255,0.279,0.554,0.46,0.629,5.315,2.213,1.106
pch_median,-5.502,-2.006,-1.381,-0.843,-0.396,0.011,0.038,0.129,0.273,0.398,0.697,0.935,1.473,1.549,1.663
close,32.78,53.2,14.86,47.23,9.2,110.51,61.95,82.73,82.99,44.58,87.93,93.05,93.74,59.96,36.57
dma,30.601,41.902,13.05,36.202,6.554,110.756,62.089,82.687,81.884,28.591,87.607,92.208,65.364,59.975,35.048
